# Conciertos

---
## Extracción de datos

Vamos a hacer un scrapping de SongKick.

Necesitamos:
- Convertir los artistas del top a su ID de songkick
- Recorrer los resultados

**0 -> LIBRERÍAS**

In [1]:
# INSTALACION

# !pip install beautifulsoup4
# !pip install requests
# !pip install lxml

# !pip install pandas
# !pip install numpy

# IMPORTACION (EJECUTAR SIEMPRE)

from bs4 import BeautifulSoup as bs
import requests

import pandas as pd
import numpy as np

import json
from time import sleep
import time
from datetime import datetime
import os

Defino las equivalencias entre el nombre en Spotify y su código en Songkick

In [2]:
URL_BASE = 'https://www.songkick.com/artists/'
GIGOGRAPHY = '/gigography?page=' # HAY QUE PONER DETRAS EL NUMERO DE PAGINA

DICT_ARTIST_WORLD = {
    # SPOTIFY - SONGKICK
    'Adele': '537914-adele',
    'Bad Bunny': '9059639-bad-bunny',
    'Becky G': '7061074-becky-g',
    'Bizarrap': '9233254-bizarrap',
    'Camila Cabello': '8504398-camila-cabello',
    'Coldplay': '197928-coldplay',
    'Cris Mj': '10173899-cris-mj',
    'Doja Cat': '8030683-doja-cat',
    'Ed Sheeran': '2083334-ed-sheeran',
    'Elley Duhé': '1236952-elley-duhe',
    'Elton John': '371163-elton-john',
    'Farruko': '3780971-farruko',
    'GAYLE': '482649-gayle',
    'Glass Animals': '1406962-glass-animals',
    'Harry Styles': '5820634-harry-styles',
    'Imagine Dragons': '2506696-imagine-dragons',
    'Jack Harlow': '8899609-jack-harlow',
    'Jaymes Young': '7001609-jaymes-young',
    'Joji': '9214309-joji',
    'Justin Bieber': '2596951-justin-bieber',
    'KAROL G': '9254044-karol-g',
    'Kate Bush': '429022-kate-bush',
    'Lil Nas X': '10001194-lil-nas-x',
    'Lizzo': '5031783-lizzo',
    'Lost Frequencies': '8423943-lost-frequencies',
    'Manuel Turizo': '9465704-manuel-turizo',
    'Olivia Rodrigo': '10112623-olivia-rodrigo',
    'OneRepublic': '568431-onerepublic',
    'Rauw Alejandro': '9920644-rauw-alejandro',
    'Ruth B.': '8675084-ruth-b',
    'Stephen Sanchez': '10218072-stephen-sanchez',
    'Tainy': '10010114-tainy',
    'The Kid LAROI': '9784044-kid-laroi',
    'The Neighbourhood': '636848-neighbourhood',
    'The Walters': '5878244-walters',
    'The Weeknd': '4363463-weeknd',
    'Tiësto': '152971-tiesto',
    'Tom Odell': '4582588-tom-odell',
}

DICT_ARTIST_SPAIN = {
    'Aitana': '9500809-aitana',
    'Alejo': '125806-alejo',
    'Ana Mena': '9106564-ana-mena',
    'Bad Bunny': '9059639-bad-bunny',
    'Bad Gyal': '8901654-bad-gyal',
    'Becky G': '7061074-becky-g',
    'Beny Jr': '10178566-beny-jr',
    'Bizarrap': '9233254-bizarrap',
    'Blessd': '10159545-blessd',
    'C. Tangana': '8692633-c-tangana',
    'Camila Cabello': '8504398-camila-cabello',
    'Cano': '249665-cano',
    'Chanel': '607616-chanel',
    'Chema Rivas': '10133758-chema-rivas',
    'Chimbala': '5588963-chimbala',
    'Chris Jedi': '10231843-chris-jedi',
    'Cris Mj': '10173899-cris-mj',
    'Daddy Yankee': '390360-daddy-yankee',
    'Danny Ocean': '9432294-danny-ocean',
    'Daviles de Novelda': '10100622-daviles-de-novelda',
    'Duki': '9235114-duki',
    'Elton John': '371163-elton-john',
    'Farruko': '3780971-farruko',
    'Feid': '2107217-feid',
    'Fuego': '267269-fuego',
    'Harry Styles': '5820634-harry-styles',
    'Justin Quiles': '8838644-justin-quiles',
    'KAROL G': '9254044-karol-g',
    'La La Love You': '975950-la-la-love-you',
    'La Pantera': '10195304-la-pantera',
    'Luar La L': '10170197-luar-la-l',
    'Manuel Turizo': '9465704-manuel-turizo',
    'Marc Seguí': '10105446-marc-segui',
    'Morad': '1252016-morad',
    'Myke Towers': '9734829-myke-towers',
    'Omar Montes': '9661509-omar-montes',
    'Oscu': '10205892-oscu',
    'Paulo Londra': '9307999-paulo-londra',
    'Polimá Westcoast': '9578634-polima-westcoast',
    'Quevedo': '10191780-quevedo',
    'Rauw Alejandro': '9920644-rauw-alejandro',
    'Rels B': '9316164-rels-b',
    'Rigoberta Bandini': '10141428-rigoberta-bandini',
    'ROSALÍA': '786955-rosalia',
    'Sebastian Yatra': '8981304-sebastian-yatra',
    'Shakira': '8981304-sebastian-yatra',
    'Tiago PZK': '10054469-tiago-pzk',
    'TINI': '1046590-tini',
    'zzoilo': '10171489-zzoilo',
}


Shows de la playlist de España

In [5]:
shows_spain = []

try:
    for spotify_name, songkick_code in DICT_ARTIST_SPAIN.items():

        print(spotify_name)

        # DEFINO LA URL BASE PARA PAGINADO
        url_request = URL_BASE + songkick_code + GIGOGRAPHY + str(1)
        
        # LA OBTENGO
        response = requests.get(url_request)
        html = response.content
        soup = bs(html, "lxml")

        try:
            last_page =  soup.find('div', class_='pagination').find('a', class_='next_page').find_previous_sibling().get('href')
            last_page = last_page.split('=')[-1]
        except:
            last_page = 0

        if last_page == 0:      
            print('Páginas: 1')
        else:
            print('Páginas: ' + str(last_page))

        for i in range(1, int(last_page)+1):
        
            # DEFINO LA URL
            url_request = URL_BASE + songkick_code + GIGOGRAPHY + str(i)
            # LA OBTENGO
            response = requests.get(url_request)
            html = response.content
            soup = bs(html, "lxml")

            # APUNTO A LA LISTA DE CONCIERTOS
            past_concerts = soup.find(id="event-listings").find('ul', class_='event-listings').find_all('li')
            

            # RECORRO LA LISTA DE CONCIERTOS
            for concert in past_concerts:

                data_show = []

                # LOCALIZO LOS EVENTOS
                try:
                    # APUNTO A LA FECHA DEL CONCIERTO
                    title = concert.get('title')

                    if '2022' in title:
                        info_json = concert.find("script", type="application/ld+json")
                        info_json = info_json.get_text(strip=True).replace("\n", "")
                        dict_evento = json.loads(info_json)[0]

                        # FECHA:
                        fecha = dict_evento['startDate']
                    
                        # UBICACION
                        ciudad = dict_evento['location']['address']['addressLocality']
                        pais = dict_evento['location']['address']['addressCountry']

                        # CREO LA FILA Y LA INSERTO EN LOS DATOS
                        data_show = [spotify_name, fecha, ciudad, pais]
                        shows_spain.append(data_show)
            
                except:
                    pass
        
        print('Añadido')
        print('-----------------------')

    print('Creando DataFrame...')
    # PASO LA BBDD A DATAFRAME
    df = pd.DataFrame(shows_spain, columns=['Track_Main_Artist', 'Fecha', 'Ciudad', 'Pais'])


    # NOTA: HAY QUE CONVERTIR LA COLUMNA 'Fecha' AL FORMATO YYYY-MM-DD
    def parse_date(str):
        # PARSEO Y UNIFICO
        try:
            fecha = datetime.strptime(str, '%Y-%m-%dT%H:%M:%S%z').strftime('%Y-%m-%d')
        except ValueError:
            fecha = datetime.strptime(str, '%Y-%m-%d')
        return fecha

    # APLICO A LA COLUMNA
    df['Fecha'] = df['Fecha'].apply(parse_date)

    df.to_csv('../data/concerts_spain.csv', sep=';', index=False)
    print('Finalizado')
    

except Exception as e:
    print("!!! ------> Error en la ejecución: " + str(e))


Aitana
Páginas: 2
Añadido
-----------------------
Alejo
Páginas: 1
Añadido
-----------------------
Ana Mena
Páginas: 1
Añadido
-----------------------
Bad Bunny
Páginas: 8
Añadido
-----------------------
Bad Gyal
Páginas: 3
Añadido
-----------------------
Becky G
Páginas: 4
Añadido
-----------------------
Beny Jr
Páginas: 1
Añadido
-----------------------
Bizarrap
Páginas: 1
Añadido
-----------------------
Blessd
Páginas: 2
Añadido
-----------------------
C. Tangana
Páginas: 2
Añadido
-----------------------
Camila Cabello
Páginas: 4
Añadido
-----------------------
Cano
Páginas: 1
Añadido
-----------------------
Chanel
Páginas: 1
Añadido
-----------------------
Chema Rivas
Páginas: 1
Añadido
-----------------------
Chimbala
Páginas: 2
Añadido
-----------------------
Chris Jedi
Páginas: 1
Añadido
-----------------------
Cris Mj
Páginas: 1
Añadido
-----------------------
Daddy Yankee
Páginas: 7
Añadido
-----------------------
Danny Ocean
Páginas: 3
Añadido
-----------------------
Daviles

C:\Users\juand\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:87: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})


Shows de la playlist mundial

In [4]:
shows_world = []

try:
    for spotify_name, songkick_code in DICT_ARTIST_WORLD.items():

        print(spotify_name)

        # DEFINO LA URL BASE PARA PAGINADO
        url_request = URL_BASE + songkick_code + GIGOGRAPHY + str(1)
        
        # LA OBTENGO
        response = requests.get(url_request)
        html = response.content
        soup = bs(html, "lxml")

        try:
            last_page =  soup.find('div', class_='pagination').find('a', class_='next_page').find_previous_sibling().get('href')
            last_page = last_page.split('=')[-1]
        except:
            last_page = 0

        if last_page == 0:      
            print('Páginas: 1')
        else:
            print('Páginas: ' + str(last_page))

        for i in range(1, int(last_page)+1):
        
            # DEFINO LA URL
            url_request = URL_BASE + songkick_code + GIGOGRAPHY + str(i)
            # LA OBTENGO
            response = requests.get(url_request)
            html = response.content
            soup = bs(html, "lxml")

            # APUNTO A LA LISTA DE CONCIERTOS
            past_concerts = soup.find(id="event-listings").find('ul', class_='event-listings').find_all('li')
            

            # RECORRO LA LISTA DE CONCIERTOS
            for concert in past_concerts:

                data_show = []

                # LOCALIZO LOS EVENTOS
                try:
                    # APUNTO A LA FECHA DEL CONCIERTO
                    title = concert.get('title')

                    if '2022' in title:
                        info_json = concert.find("script", type="application/ld+json")
                        info_json = info_json.get_text(strip=True).replace("\n", "")
                        dict_evento = json.loads(info_json)[0]

                        # FECHA:
                        fecha = dict_evento['startDate']
                    
                        # UBICACION
                        ciudad = dict_evento['location']['address']['addressLocality']
                        pais = dict_evento['location']['address']['addressCountry']

                        # CREO LA FILA Y LA INSERTO EN LOS DATOS
                        data_show = [spotify_name, fecha, ciudad, pais]
                        shows_world.append(data_show)
            
                except:
                    pass
        
        print('Añadido')
        print('-----------------------')

    print('Creando DataFrame...')
    # PASO LA BBDD A DATAFRAME
    df = pd.DataFrame(shows_world, columns=['Track_Main_Artist', 'Fecha', 'Ciudad', 'Pais'])

    # NOTA: HAY QUE CONVERTIR LA COLUMNA 'Fecha' AL FORMATO YYYY-MM-DD
    def parse_date(str):
        # PARSEO Y UNIFICO
        try:
            fecha = datetime.strptime(str, '%Y-%m-%dT%H:%M:%S%z').strftime('%Y-%m-%d')
        except ValueError:
            fecha = datetime.strptime(str, '%Y-%m-%d')
        return fecha

    # APLICO A LA COLUMNA
    df['Fecha'] = df['Fecha'].apply(parse_date)


    df.to_csv('../data/concerts_world.csv', sep=';', index=False)
    print('Finalizado')
    

except Exception as e:
    print("!!! ------> Error en la ejecución: " + str(e))


Adele
Páginas: 7
Añadido
-----------------------
Bad Bunny
Páginas: 8
Añadido
-----------------------
Becky G
Páginas: 4
Añadido
-----------------------
Bizarrap
Páginas: 1
Añadido
-----------------------
Camila Cabello
Páginas: 4
Añadido
-----------------------
Coldplay
Páginas: 23
Añadido
-----------------------
Cris Mj
Páginas: 1
Añadido
-----------------------
Doja Cat
Páginas: 3
Añadido
-----------------------
Ed Sheeran
Páginas: 21
Añadido
-----------------------
Elley Duhé
Páginas: 1
Añadido
-----------------------
Elton John
Páginas: 76
Añadido
-----------------------
Farruko
Páginas: 6
Añadido
-----------------------
GAYLE
Páginas: 2
Añadido
-----------------------
Glass Animals
Páginas: 12
Añadido
-----------------------
Harry Styles
Páginas: 5
Añadido
-----------------------
Imagine Dragons
Páginas: 15
Añadido
-----------------------
Jack Harlow
Páginas: 6
Añadido
-----------------------
Jaymes Young
Páginas: 3
Añadido
-----------------------
Joji
Páginas: 2
Añadido
--------

C:\Users\juand\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:86: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
